# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.dropna()
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,namibe,85,AO,1578408217,84,-15.19,12.15,73.47,9.24
1,1,port lincoln,0,AU,1578408217,66,-34.72,135.86,66.76,10.56
2,2,ribeira grande,40,PT,1578408217,88,38.52,-28.70,66.20,13.87
3,3,lincoln,75,GB,1578408072,87,53.23,-0.54,52.00,20.80
4,4,deputatskiy,100,RU,1578408218,92,69.30,139.90,-6.23,2.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:

# Configure gmaps
gmaps.configure(api_key=g_key)

locations= cities_df[["Lat","Lng"]]


# Plot Heatmap


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=cities_df["Humidity"],dissipating=False,max_intensity=10,point_radius=1)
fig = gmaps.figure()

# Add layer
fig.add_layer(heat_layer)
# Display figure
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
#A max temperature lower than 80 degrees but higher than 70.  * Wind speed less than 10 mph.  * Zero cloudiness.

hotel_df = cities_df.loc[((cities_df["Max Temp"] <80)  &  
                         (cities_df["Max Temp"] >70))&
                                                 (cities_df["Wind Speed"] < 10) &
                                                 (cities_df["Cloudiness"] == 0) ]

#hotel_df.dropna()
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
59,59,pyapon,0,MM,1578408221,64,16.29,95.68,72.57,2.06
115,115,chuy,0,UY,1578408224,34,-33.69,-53.46,79.41,9.93
258,258,acajutla,0,SV,1578408234,64,13.59,-89.83,72.00,2.24
342,342,antofagasta,0,CL,1578407989,64,-23.65,-70.40,71.60,9.17
432,432,santa vitoria do palmar,0,BR,1578408246,34,-33.52,-53.37,79.41,9.93
440,440,bella union,0,CO,1578408246,56,8.50,-73.35,77.85,3.13
526,526,ovalle,0,CL,1578407962,40,-30.60,-71.20,73.65,7.72
528,528,ipswich,0,AU,1578408252,73,-27.62,152.76,75.99,4.70
533,533,bilimora,0,IN,1578408253,65,20.77,72.97,77.00,7.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:

hotel_df["Hotel Name"] = " "
hotel_df

C:\Users\badda\Anaconda3\envs\PythhonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
59,59,pyapon,0,MM,1578408221,64,16.29,95.68,72.57,2.06,
115,115,chuy,0,UY,1578408224,34,-33.69,-53.46,79.41,9.93,
258,258,acajutla,0,SV,1578408234,64,13.59,-89.83,72.00,2.24,
342,342,antofagasta,0,CL,1578407989,64,-23.65,-70.40,71.60,9.17,
432,432,santa vitoria do palmar,0,BR,1578408246,34,-33.52,-53.37,79.41,9.93,
440,440,bella union,0,CO,1578408246,56,8.50,-73.35,77.85,3.13,
526,526,ovalle,0,CL,1578407962,40,-30.60,-71.20,73.65,7.72,
528,528,ipswich,0,AU,1578408252,73,-27.62,152.76,75.99,4.70,
533,533,bilimora,0,IN,1578408253,65,20.77,72.97,77.00,7.14,


In [35]:
params = {
    "radius" : 50000,
    "types": "Hotel",
    "keyword": "Lodging",
    "key": g_key
}

In [36]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Lat']
    lng = row['Lng']

    # update location key value
    params['location'] = f"{lat},{lng}"
    # make request
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
   
   
hotel_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
59,59,pyapon,0,MM,1578408221,64,16.29,95.68,72.57,2.06,Shoan advertising
115,115,chuy,0,UY,1578408224,34,-33.69,-53.46,79.41,9.93,Bella Bungalows
258,258,acajutla,0,SV,1578408234,64,13.59,-89.83,72.00,2.24,Hotel Palo Verde
342,342,antofagasta,0,CL,1578407989,64,-23.65,-70.40,71.60,9.17,Hotel Terrado Suites Antofagasta
432,432,santa vitoria do palmar,0,BR,1578408246,34,-33.52,-53.37,79.41,9.93,Pousada Quebra Mar
440,440,bella union,0,CO,1578408246,56,8.50,-73.35,77.85,3.13,HOTEL LOS CASTILLOS
526,526,ovalle,0,CL,1578407962,40,-30.60,-71.20,73.65,7.72,Hacienda Los Andes
528,528,ipswich,0,AU,1578408252,73,-27.62,152.76,75.99,4.70,Mt Glorious Getaways
533,533,bilimora,0,IN,1578408253,65,20.77,72.97,77.00,7.14,Mirasol Resort


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))